In [7]:
import numpy as np
import geopandas as gpd
from shapely.geometry import box
from room import Room

In [4]:
rect = box(0, 0, 2, 1)
gdf = gpd.GeoDataFrame({'name': ['rect'], 'type': ['room'], 'geometry': [rect]})

In [5]:
height = 0.1
min_intensity = 1

In [8]:
room = Room(gdf,
            guard_scale=0.8,
            room_res=200,
            guard_res=200)

In [152]:
# Branch and bound algorithm: divide the rectangle up into subrectangles,
# splitting along the longest side. Lower/upper bound the illumination
# at each rectangle by determining illumination at the corners and taking
# min/max illuminations.
def box_intensities(room, weights, box):
    minx, miny, maxx, maxy = box
    h = height
    intensities = np.zeros((4, room.guard_grid.shape[0]))
    gx = room.guard_grid[:, 0]
    gy = room.guard_grid[:, 1]
    intensities[0] = weights / (((minx - gx) ** 2) + ((miny - gy) ** 2) + h)
    intensities[1] = weights / (((minx - gx) ** 2) + ((maxy - gy) ** 2) + h)
    intensities[2] = weights / (((maxx - gx) ** 2) + ((miny - gy) ** 2) + h)
    intensities[3] = weights / (((maxx - gx) ** 2) + ((maxy - gy) ** 2) + h)
    return intensities

def illumination_lower_bound(room, weights, box):
    return np.sum(np.min(box_intensities(room, weights, box), axis=0))
    
def illumination_upper_bound(room, weights, box):
    minx, miny, maxx, maxy = box
    distances = np.zeros(room.guard_grid.shape[0])
    for idx, (x, y) in enumerate(room.guard_grid):
        if minx <= x <= maxx and miny <= y <= maxy:
            # Case: guard point within box.
            distances[idx] = 0
        elif minx <= x <= maxx:
            # Case: guard point above or below box.
            distances[idx] = min([abs(y - miny), abs(y - maxy)])**2
        elif miny <= y <= maxy:
            # Case: guard point to the left or right of box.
            distances[idx] = min([abs(x - minx), abs(x - maxx)])**2
        else:
            distances[idx] = min([
                (minx - x)**2 + (miny - y)**2,
                (minx - x)**2 + (maxy - y)**2,
                (maxx - x)**2 + (miny - y)**2,
                (maxx - x)**2 + (maxy - y)**2,
            ])
    distances += height
    return np.sum(weights / distances)
            
def branch_bound_rect(room, weights, max_iters=100, epsilon=1e-5):
    rects = [room.room.bounds]
    lbs = [illumination_lower_bound(room, weights, rects[0])]
    ubs = [illumination_upper_bound(room, weights, rects[0])]
    L_idx = 0
    Q = rects[0]
    L = lbs[0]
    U = ubs[0]
    
    for i in range(max_iters):
        print(i, U - L)
        if U - L <= epsilon:
            break
        # Split Q along its longest side and replace with Q1, Q2.
        minx, miny, maxx, maxy = Q
        width = maxx - minx
        height = maxy - miny
        if width > height:
            Q1 = (minx, miny, minx + (maxx - minx) / 2, maxy)
            Q2 = (minx + (maxx - minx) / 2, miny, maxx, maxy)
        else:
            Q1 = (minx, miny, maxx, miny + (maxy - miny) / 2)
            Q2 = (minx, miny + (maxy - miny) / 2, maxx, maxy)
        rects[L_idx] = Q1
        rects.append(Q2)
        lbs[L_idx] = illumination_lower_bound(room, weights, Q1)
        lbs.append(illumination_lower_bound(room, weights, Q2))
        ubs[L_idx] = illumination_upper_bound(room, weights, Q1)
        ubs.append(illumination_upper_bound(room, weights, Q2))
        # Update bounds.
        L_idx = np.argmin(lbs)
        Q = rects[L_idx]
        L = np.min(lbs)
        U = np.min(ubs)
    return Q, L, U

In [153]:
random_weights = np.random.random(room.guard_grid.shape[0])

In [154]:
branch_bound_rect(room, random_weights)

0 1065.064451899336
1 744.11868475216
2 639.5337807062734
3 628.7143144898422
4 344.0740196260071
5 342.6786034982231
6 341.0761950068785
7 331.6273743377777
8 221.0528724556621
9 220.8715549000201
10 125.10191116367582
11 86.35899503776926
12 49.25722619088691
13 36.04769669036722
14 21.597931237168723
15 16.344219022449266
16 10.117770652563891
17 7.782982347499399
18 4.898948286536012
19 3.798439009723694
20 2.410782841628503
21 1.8764931612408589
22 1.1958789474219884
23 0.932632204254773
24 0.5955812917911771
25 0.46492090676062503
26 0.2972035849571739
27 0.2321127039331543
28 0.1484553373189641
29 0.1159695457424732
30 0.0741910935974488
31 0.057963087704877125
32 0.03708640787220929
33 0.028976124609386034
34 0.018540919809225898
35 0.014486707749995276
36 0.009269888948480798
37 0.0072430152683296
38 0.0046348017446575795
39 0.0036214229864839353
40 0.002317365191117915
41 0.0018106903318226841
42 0.0011586736754054527
43 0.0009053398756293518
44 0.000579334607678561
45 0.0004

((0.0, 0.0, 2.9802322387695312e-08, 2.9802322387695312e-08),
 106.40984154213191,
 106.40985059420086)